In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import *
import os


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trying_delta")\
.config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0")\
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
.getOrCreate()

23/11/01 22:23:44 WARN Utils: Your hostname, dj resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp0s20f3)
23/11/01 22:23:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/deepak/Downloads/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/deepak/.ivy2/cache
The jars for the packages stored in: /home/deepak/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cd3c0e2a-943d-4277-8537-2ea96489ef01;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 539ms :: artifacts dl 24ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

23/11/01 22:23:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [21]:
ip='src/input.csv'
op='op/'
nK=["id"]
hashCol="SHA256"
sKey="sK"

In [4]:
df = (spark
        .read
        .format('csv')
        .option('header', 'true')
        .option('inferSchema', 'true')
        .load(ip))

df.show()

+---+----+----+
| id|name| age|
+---+----+----+
|  1|   X|  10|
|  2|   B|  11|
|  3|   C|null|
|  4|null|   5|
|  5|   D|   6|
+---+----+----+



In [5]:
def handleIntegerType(df:DataFrame,cols:list[str])->DataFrame:
    return df.na.fill(0,cols)

def handleStringType(df:DataFrame,cols:list[str])->DataFrame:
    df=df.na.fill("NA",cols)
    for c in cols:
        df=df.withColumn(c,trim(df[c]))
    return df
        
def cleanDF(df:DataFrame)->DataFrame:
    d={}
    for field in df.schema:
        fieldType=str(field.dataType).replace("()","")
        if((fieldType in d.keys()) == False):
            d[fieldType]=[field.name]
        else:
            d[fieldType].append(field.name)

    for key,cols  in d.items():
        if key=='IntegerType': df=df.transform(handleIntegerType,cols);
        elif key=='StringType': df=df.transform(handleStringType,cols);
    return df

In [6]:
df=df.transform(cleanDF)
df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|   X| 10|
|  2|   B| 11|
|  3|   C|  0|
|  4|  NA|  5|
|  5|   D|  6|
+---+----+---+



In [27]:
def addHash(df:DataFrame,nK:list[str])->DataFrame:
    df= df.withColumn(hashCol,sha2(concat_ws("_",*nK),256))
    return df

def addSkey(df:DataFrame)->DataFrame:
    return df.withColumn(sKey,row_number().over(Window.orderBy(hashCol)))

def getDeltaDF():
    if os.path.exists(op):
        deltaDF=DeltaTable.forPath(spark,op)
        return deltaDF
    return 0

def fullLoad(df:DataFrame,nK:list[str]):
   df=df.transform(addHash,nK)
   return df.transform(addSkey) 

def deltaLoad(srdDF:DataFrame,deltaDF:DeltaTable,nK:list[str]): 
    tempDeltaDF=deltaDF.toDF().select(hashCol,sKey)
    maxKey=tempDeltaDF.select(max(sKey)).first()[f"max({sKey})"]

    srcDF=srdDF.transform(addHash,nK)
    fullDF=srcDF.join(tempDeltaDF,[hashCol],"full")
    newRecordsDF=fullDF.filter("sk is null").transform(addSkey).withColumn(sKey,maxKey+col(sKey)+1)
    fullDF=fullDF.filter("sk is not null").union(newRecordsDF)
    # fullDF.show()
    return fullDF
    
    # if(newRecordsDF.count()==0):print(f" No records with new naturalKeys {nK}, Checking for Updated Records  ")
    # else:
    #     newRecordsDF=newRecordsDF.transform(addSkey)
    #     maxKey=deltaDF.toDF().select(max(sKey)).first()[f"max({sKey})"]
    #     newRecordsDF=newRecordsDF.withColumn(sKey,df.sKey+maxKey+1)

    

In [28]:
deltaDF=getDeltaDF()

if deltaDF==0:
    resDF=fullLoad(df,nK)
    resDF.write.format("delta").save(op)
else:
    resDF=deltaLoad(df,deltaDF,nK)


23/11/01 22:46:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:46:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:46:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:46:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:46:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:46:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 2

In [26]:
deltaLoad(df,deltaDF,nK)

23/11/01 22:45:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:45:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:45:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:45:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:45:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 22:45:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/11/01 2